In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
# Load your model from Hugging Face Hub
model_name = "google/mt5-small"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [5]:
def translate(text: str, source_lang: str, target_lang: str, max_length: int = 128) -> str:
    input_text = f"translate {source_lang} to {target_lang}: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_length).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            num_beams=4,
            early_stopping=True
        )

    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated




In [6]:
# English → French
en_text = "Comment êtes-vous aujourd'hui ?"
fr_translation = translate(en_text, "French",  "English")
print("FR → EN:", fr_translation)

FR → EN: <extra_id_0>


In [7]:
from datasets import load_dataset

In [8]:
# ===============================
# Load OPUS-100 en-fr dataset
# ===============================
dataset = load_dataset("opus100", "en-fr")["train"].select(range(100))

README.md: 0.00B [00:00, ?B/s]

en-fr/test-00000-of-00001.parquet:   0%|          | 0.00/327k [00:00<?, ?B/s]

en-fr/train-00000-of-00001.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

en-fr/validation-00000-of-00001.parquet:   0%|          | 0.00/334k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
# ===============================
# Translate and collect outputs
# ===============================
source_texts = dataset["translation"]
reference_english_sentences = [item["en"] for item in source_texts]
french_sentences = [item["fr"] for item in source_texts]

In [10]:
predicted_english_sentences = []

In [11]:
print("Translating 100 French sentences using fine-tuned mT5...\n")

Translating 100 French sentences using fine-tuned mT5...



In [12]:
from tqdm import tqdm

In [13]:
for sent in tqdm(french_sentences):
    translated = translate(sent, "French","English")
    predicted_english_sentences.append(translated)

100%|██████████| 100/100 [00:10<00:00,  9.56it/s]


In [14]:
# ===============================
# Show predictions vs references
# ===============================
for i, (pred, ref) in enumerate(zip(predicted_english_sentences,reference_english_sentences), start=1):
    print(f"\nExample {i}")
    print(f"English   : {french_sentences[i-1]}")
    print(f"Reference : {ref}")
    print(f"Predicted : {pred}")


Example 1
English   : The time now is 05:05 .
Reference : The time now is 05:08 .
Predicted : <extra_id_0>

Example 2
English   : Le présent règlement entre en vigueur le septième jour suivant celui de sa publication au Journal officiel de l'Union européenne.
Reference : This Regulation shall enter into force on the seventh day following its publication in the Official Journal of the European Union.
Predicted : <extra_id_0>.

Example 3
English   : Qu'est-ce que c'est que ça ?
Reference : Hello, what's that?
Predicted : <extra_id_0>

Example 4
English   : Et alors, je t'apprendrai tout ce que je sais.
Reference : And then I will teach you everything i know.
Predicted : <extra_id_0>

Example 5
English   : Par ici !
Reference : Did you find something?
Predicted : <extra_id_0>

Example 6
English   : Article 6
Reference : Article 6
Predicted : <extra_id_0>

Example 7
English   : - Tu n'es pas responsable.
Reference : Oh, honey, it's not your fault.
Predicted : <extra_id_0>

Example 8
Engli

In [15]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00


In [16]:
import sacrebleu

In [17]:
# ===============================
# Compute BLEU score
# ===============================
references = [reference_english_sentences]  # correct format
bleu = sacrebleu.corpus_bleu(predicted_english_sentences, references)

In [18]:
print("\n===== BLEU Score =====")
print(f"BLEU: {bleu.score:.2f}")


===== BLEU Score =====
BLEU: 0.10


In [19]:
# chrF
chrf = sacrebleu.corpus_chrf(predicted_english_sentences, references)
print("Corpus chrF score:", chrf.score)

Corpus chrF score: 2.460643345609764


In [20]:
# TER
ter = sacrebleu.corpus_ter(predicted_english_sentences, references)
print("Corpus TER score:", ter.score)

Corpus TER score: 100.0
